In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os

%matplotlib inline

In [5]:
# Data Augmentation and Normalization
data_transforms = {
    "train": transforms.Compose([
        transforms.Scale(266),
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Scale(266),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
from data_loaders import ClothingAttributesDataset, combine_labels

In [7]:
IMAGES_FOLDER = "data/ClothingAttributeDataset/images/"
LABEL_DIR = "data/ClothingAttributeDataset/labels/"
labels_df = combine_labels(LABEL_DIR, na_value=0.5)
# labels_df.to_csv("data/labels.csv", index=False)

In [8]:
# Map Target Column with their index and Output Dimensions
TARGET_COLUMNS = dict([(column, (idx, 1)) for idx, column in enumerate(labels_df.columns)])
TARGET_COLUMNS["category_GT"] = (TARGET_COLUMNS["category_GT"][0], 7)
TARGET_COLUMNS["sleevelength_GT"] = (TARGET_COLUMNS["sleevelength_GT"][0], 3)
TARGET_COLUMNS["neckline_GT"] = (TARGET_COLUMNS["neckline_GT"][0], 3)

In [9]:
target_columns = [column for column in labels_df.columns 
                  if column not in ["category_GT", "sleevelength_GT", "neckline_GT"]]

In [10]:
labels_df.columns

Index(['white_GT', 'skin_exposure_GT', 'pattern_spot_GT', 'pattern_solid_GT',
       'neckline_GT', 'collar_GT', 'yellow_GT', 'many_colors_GT', 'gray_GT',
       'pattern_graphics_GT', 'sleevelength_GT', 'gender_GT', 'scarf_GT',
       'category_GT', 'pattern_plaid_GT', 'green_GT', 'necktie_GT', 'blue_GT',
       'pattern_stripe_GT', 'pattern_floral_GT', 'brown_GT', 'red_GT',
       'cyan_GT', 'placket_GT', 'black_GT', 'purple_GT'],
      dtype='object')

In [11]:
binary_columns = ['skin_exposure_GT', 'collar_GT', 'gender_GT', 'scarf_GT', 'necktie_GT',  'placket_GT']
multi_columns = ['category_GT', 'neckline_GT', 'sleevelength_GT']
    
color_columns = ['white_GT', 'yellow_GT', 'gray_GT', 'green_GT', 'blue_GT', 'brown_GT', 'red_GT',
                 'cyan_GT', 'black_GT', 'purple_GT', 'many_colors_GT']
pattern_columns = ['pattern_spot_GT', 'pattern_solid_GT', 'pattern_graphics_GT', 
                  'pattern_plaid_GT', 'pattern_stripe_GT', 'pattern_floral_GT']

In [12]:
target_columns = binary_columns

In [13]:
len(binary_columns + multi_columns + color_columns + pattern_columns)

26

In [14]:
len(target_columns)

6

In [15]:
# DATA_DIR = "/Users/sampathweb/datasets/hymenoptera_data/"
# dsets = {
#     "train": ClothingAttributesDataset(os.path.join(DATA_DIR, "train"), labels_df,
#                                   data_transforms["train"]),
#     "val": datasets.ImageFolder(os.path.join(DATA_DIR, "val"),
#                                data_transforms["val"])

dsets = {
    "train": ClothingAttributesDataset(IMAGES_FOLDER, labels_df, target_columns=target_columns,
                                       transform=data_transforms["train"]),
    "val": ClothingAttributesDataset(IMAGES_FOLDER, labels_df, target_columns=target_columns,
                                     transform=data_transforms["val"])    
}

In [16]:
dset_sizes = {
    "train": len(dsets["train"]),
    "val": len(dsets["val"])
}

In [17]:
dset_sizes

{'train': 1856, 'val': 1856}

In [18]:
batch_size = 256
num_workers = 4

In [19]:
dset_loaders = {
    "train": torch.utils.data.DataLoader(dsets["train"], batch_size=batch_size,
                                        shuffle=True, num_workers=num_workers),
    "val": torch.utils.data.DataLoader(dsets["val"], batch_size=batch_size,
                                      shuffle=True, num_workers=num_workers)
}

In [22]:
len(dset_loaders["train"].dataset)

1856

In [20]:
print(dir(dset_loaders["train"]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'batch_size', 'collate_fn', 'dataset', 'num_workers', 'pin_memory', 'sampler']


In [126]:
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
# Parameter of newly constructed modules have requires_grad=True by default
num_features = model.fc.in_features
# Take 23 Cloting Attributes
model.fc = nn.Sequential(*[nn.Linear(num_features, len(target_columns)), nn.Sigmoid()])

use_gpu = torch.cuda.is_available()
if use_gpu:
    model = model.cuda()

criterion = nn.BCELoss()

Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f7340aa1208>>
Traceback (most recent call last):
  File "/home/paperspace/miniconda3/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 218, in __del__
    self._shutdown_workers()
  File "/home/paperspace/miniconda3/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 214, in _shutdown_workers
    self.index_queue.put(None)
  File "/home/paperspace/miniconda3/envs/py36/lib/python3.6/multiprocessing/queues.py", line 355, in put
    self._writer.send_bytes(obj)
  File "/home/paperspace/miniconda3/envs/py36/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/paperspace/miniconda3/envs/py36/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/paperspace/miniconda3/envs/py36/lib/python3.6/

In [24]:
np.isnull

AttributeError: module 'numpy' has no attribute 'isnull'

In [127]:
def optim_scheduler_ft(model, epoch, init_lr=0.001, lr_decay_epoch=7):
    lr = init_lr * (0.1**(epoch//lr_decay_epoch))
    
    if epoch % lr_decay_epoch == 0:
        print("LR is set to {}".format(lr))
        
    optimizer = optim.SGD(model.fc.parameters(), lr=lr, momentum=0.9)
    return optimizer

In [131]:
def train_model(model, criterion, optim_scheduler, num_epochs=25):
    since = time.time()
    best_model = model
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs - 1))
        
        # Each epoch has a train and validation Phase
        for phase in ["train"]:  # "val"
            if phase == "train":
                optimizer = optim_scheduler(model, epoch)
            
            running_loss = 0.0
            running_corrects = 0.0
            
            # Iterate over data
            for data in dset_loaders[phase]:
                # Get the inputs
                inputs, labels = data
            
                labels = labels.float()
                num_targets = labels.size()[1]
                
                # Wrap them in Variable
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), \
                                             Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward
                outputs = model(inputs)
                preds = torch.round(outputs)
                total_loss = sum(criterion(outputs[:, col], labels[:, col]) for col in range(num_targets))
#                 _, preds = torch.max(outputs.data, 1)
#                 loss = criterion(outputs, labels)
                
                # Backward + Optimize only in Training Phase
                if phase == "train":
                    total_loss.backward()
                    optimizer.step()
                    
                # Statistics
                running_loss += total_loss.data[0]
                running_corrects += torch.sum(preds.data == labels.data) / num_targets # Caveat 0.5 is coverted to Ones
                
            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects / dset_sizes[phase]
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            # Deep copy the model
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model)
                
    time_elapsed = time.time() - since
    print("Training completed in {:0f}m {:0f}s".format(
        time_elapsed // 60, time_elapsed % 60))
    print("Best val Acc: {:4f}".format(best_acc))
    return best_model

In [132]:
dset_sizes["train"]

1856

In [134]:
model = train_model(model, criterion, optim_scheduler_ft, num_epochs=10)

Epoch 0/9
LR is set to 0.001
train Loss: 0.0138 Acc: 0.7007
Epoch 1/9
train Loss: 0.0136 Acc: 0.7027
Epoch 2/9
train Loss: 0.0134 Acc: 0.7140
Epoch 3/9
train Loss: 0.0132 Acc: 0.7220
Epoch 4/9
train Loss: 0.0130 Acc: 0.7259
Epoch 5/9
train Loss: 0.0129 Acc: 0.7328
Epoch 6/9
train Loss: 0.0127 Acc: 0.7361
Epoch 7/9
LR is set to 0.0001
train Loss: 0.0125 Acc: 0.7419
Epoch 8/9
train Loss: 0.0126 Acc: 0.7399
Epoch 9/9
train Loss: 0.0126 Acc: 0.7397
Training completed in 1.000000m 16.836530s
Best val Acc: 0.000000


In [113]:
tot_loss

[Variable containing:
  0.3510
 [torch.cuda.FloatTensor of size 1 (GPU 0)], Variable containing:
  0.5304
 [torch.cuda.FloatTensor of size 1 (GPU 0)], Variable containing:
  0.5205
 [torch.cuda.FloatTensor of size 1 (GPU 0)], Variable containing:
  0.4167
 [torch.cuda.FloatTensor of size 1 (GPU 0)], Variable containing:
  0.3537
 [torch.cuda.FloatTensor of size 1 (GPU 0)], Variable containing:
  0.5233
 [torch.cuda.FloatTensor of size 1 (GPU 0)]]

In [114]:
losses

Variable containing:
 2.6956
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [107]:
loss1.data[0]

3.131206512451172

## Validate the Results

In [93]:
for data in dset_loaders["train"]:
    inputs, labels = data
    if use_gpu:
        X = Variable(inputs.cuda())
    else:
        X = Variable(inputs)
    results = model(X)
    if use_gpu:
        results = results.cpu()
    break

In [94]:
labels_arr = labels.numpy()

In [95]:
labels_arr.size

1536

In [96]:
results_arr = results.data.numpy()

In [97]:
from sklearn.metrics import confusion_matrix

In [98]:
confusion_matrix(np.round(results_arr.flatten()).astype(int), labels_arr.flatten().astype(int))

array([[822, 130],
       [198, 386]])

In [ ]:
reset = torchvision.models.resnet18(pretrained=True)
for param in resnet.parameters():
    param.requires_grad = False
    
# # Parameter of newly constructed modules have requires_grad=True by default
# num_features = model.fc.in_features
# # Take 23 Cloting Attributes
# model.fc = nn.Sequential(*[nn.Linear(num_features, len(target_columns)), nn.Sigmoid()])

# use_gpu = torch.cuda.is_available()
# if use_gpu:
#     model = model.cuda()

# criterion = nn.BCELoss()

In [ ]:
pretrained_model = nn.Sequential(*list(resnet.children())[:-1])


( 0 , 0 ,.,.) = 
 -0.1828 -0.5424 -0.7650  ...   0.1597  0.6392  1.0159
 -0.6623 -0.7993 -0.6794  ...  -0.1314  0.4337  0.9646
 -0.7993 -0.5424 -0.2171  ...  -0.1999  0.3309  0.9132
           ...             ⋱             ...          
 -0.4397 -0.2342 -0.4226  ...  -0.2513 -0.2171 -0.3027
 -0.3198 -0.3883 -0.5082  ...  -0.5938 -0.3541 -0.4054
 -0.2513 -0.1999 -0.5253  ...  -0.5596 -0.6109 -0.4054

( 0 , 1 ,.,.) = 
  0.7654  0.5378  0.3277  ...   0.0651  0.4153  0.7304
  0.3803  0.3102  0.4503  ...  -0.1450  0.2927  0.7304
  0.2227  0.4678  0.7654  ...  -0.2150  0.2052  0.6779
           ...             ⋱             ...          
 -0.2500 -0.0399 -0.1800  ...   0.0126  0.0651 -0.0049
 -0.1625 -0.2150 -0.3200  ...  -0.3550 -0.1275 -0.1450
 -0.1099 -0.0224 -0.3375  ...  -0.3725 -0.3550 -0.2325

( 0 , 2 ,.,.) = 
  0.3568  0.0431 -0.1661  ...  -0.1312  0.1999  0.4788
 -0.0964 -0.2184 -0.0964  ...  -0.3578  0.0431  0.4439
 -0.2707 -0.0441  0.2871  ...  -0.4275 -0.0615  0.3568
           

In [175]:
f = resnet(Variable(inputs))

RuntimeError: Variable data has to be a tensor, but got Variable

In [167]:
f = f.view(f.size(0), -1)

In [ ]:
outputs = outputs.view(0, -1)

In [168]:
model = nn.Sequential(nn.Linear(512, 3))

In [177]:
if use_gpu:
    resnet.cuda()
    pretrained_model = nn.Sequential(*list(resnet.children())[:-1])

In [179]:
for data in dset_loaders["train"]:
    # Get the inputs
    inputs, labels = data

    labels = labels.float()
    num_targets = labels.size()[1]

    # Wrap them in Variable
    if use_gpu:
        inputs, labels = Variable(inputs.cuda()), \
                                 Variable(labels.cuda())
    else:
        inputs, labels = Variable(inputs), Variable(labels)

    # Forward
    outputs = resnet(inputs)

RuntimeError: cuda runtime error (2) : out of memory at /data/users/soumith/miniconda2/conda-bld/pytorch-cuda80-0.1.10_1488758793045/work/torch/lib/THC/THCGeneral.c:689